In [1]:
!pip show dlt

Name: dlt
Version: 1.13.0
Summary: dlt is an open-source python-first scalable data loading library that does not require any backend to run.
Home-page: https://github.com/dlt-hub
Author: 
Author-email: "dltHub Inc." <services@dlthub.com>
License-Expression: Apache-2.0
Location: /Users/grethel/Library/Caches/pypoetry/virtualenvs/llm-zoomcamp-glen-EQX3Pm9H-py3.12/lib/python3.12/site-packages
Requires: click, fsspec, gitpython, giturlparse, hexbytes, humanize, jsonpath-ng, orjson, packaging, pathvalidate, pendulum, pluggy, pytz, pyyaml, requests, requirements-parser, rich-argparse, semver, setuptools, simplejson, sqlglot, tenacity, tomlkit, typing-extensions, tzdata
Required-by: 


In [5]:
import dlt
import requests
from dlt.destinations import qdrant

In [4]:
@dlt.resource(name="zoomcamp_data")
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

### Question 2. dlt pipeline

In [6]:
qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

In [7]:
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

Run started at 2025-07-10 00:19:21.310128+00:00 and COMPLETED in 10.99 seconds with 4 steps.
Step extract COMPLETED in 1.36 seconds.

Load package 1752106768.3488178 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.07 seconds.
Normalized data for the following tables:
- zoomcamp_data: 948 row(s)
- _dlt_pipeline_state: 1 row(s)

Load package 1752106768.3488178 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 2.53 seconds.
Pipeline zoomcamp_pipeline load step completed in 2.52 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /Users/grethel/Documents/projects/llm-zoomcamp-glen/workshops/db.qdrant location to store data
Load package 1752106768.3488178 is LOADED and contains no failed jobs

Step run COMPLETED in 10.99 seconds.
Pipeline zoomcamp_pipeline load step completed in 2.52 seconds
1 load packa